In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
import json

In [ ]:
# data structure: names of model directories and number of executions for each model
# to build the paths to all of the model directories
targets = [
    ['166404_083202_043201_TPU_10', 10],
    ['086404_043202_043201_TPU_11', 10],
    ['086404_043202_043201_TPU_13', 10],
    ['086404_086402_043201_TPU_14', 10],
    ['086404_086402_043201_TPU_15', 10],
    ['086403_083202_043201_TPU_16', 10],
    ['086401_083201_043201_TPU_17', 10],    
    ['086404_086404_043202_TPU_18', 10],
    ['086404_086404_043202_TPU_19', 10],
    ['086404_086404_043202_TPU_20', 10],
    ['086404_086404_046404_TPU_21', 10],
    ['086404_086404_043202_TPU_22', 10],
    ['086404_086404_043202_TPU_23', 10],
    ['086404_086404_043202_TPU_24', 10],
    ['089604_089604_043202_TPU_25', 10],
    ['084804_084804_043202_TPU_26', 10],
    ['0812804_0812804_046402_TPU_27', 10],
    ['0812804_0812804_046402_TPU_30', 10],
    ['0812804_0812804_046402_TPU_31', 10],
    ['0812804_0812804_046402_TPU_32', 10],
    ['0812804_0812804_046402_TPU_33', 10],
    ['0812804_0812804_046402_TPU_34', 10],
    ['0812804_0812804_046402_TPU_35', 10],
    ['0812804_0812804_046402_TPU_36', 9],
    ['0812804_0812804_046402_TPU_37', 10],
    ['0812804_0812804_046402_TPU_38', 10],
    ['0812804_0812804_046402_TPU_39', 10],
    ['0812804_0812804_046402_TPU_40', 10],
    ['0812804_0812804_046402_TPU_41', 10],
    ['0812804_0812804_046402_TPU_42', 10],
    ['0812804_0812804_046402_TPU_50', 30],
    ['0812804_0812804_046402_TPU_51', 30],
    ['0812804_0812804_046402_TPU_52', 20],
]

### build the individual paths to all of the model directories
### point to prediction_results_on_test_dataset.json
### and to prediction_results_on_eval_dataset.json

### generate a bash script to copy from Google Storage to local disk on stats/ directory

for target in targets:
    for counter in np.arange(target[1]):
        for dataset in ['test', 'eval']:
            
            source_path = 'gs://cbidmltsf/models/{}_{:02d}/prediction_results_on_{}_dataset.json'.\
            format(target[0], counter, dataset)
            
            destination_path = 'stats/{}_{:02d}/prediction_results_on_{}_dataset.json'.\
            format(target[0][-6:], counter, dataset)
            
            print('gsutil cp {0} {1}'.format(source_path, destination_path))

In [7]:
# print output from previous cell was used to download prediction json files
# from model directories in GS to the local station, in stats/ directory
# now build a list with those local json files to iterate on it

In [ ]:
# UPDATE: USE GOOGLE STORAGE TO BACKUP AND TO TRANSFER BETWEEN STATIONS VIA RSYNC
# THERE IS NO NEED TO DOWNLOAD ANYMORE, JUST SYNC THE STATS/ AND DATABASE/FOLDERS

In [8]:
# keys in predictions json files are: ['predictions', 'string_timestamps', 'targets']

In [9]:
# transfer prediction results from JSON files to a Pandas dataframe
# required columns in the dataframe are:
# model_id: string, a simple identifer for the model directory
# string_timestamp
# prediction
# add, eventually, columns for the parameter values
# embedding dimensions[mh, md, mw]
# architecture [[hh1, hh2, ..., hhn], [hd1, hd2, ..., hdn], [hw1, hw2, ..., hwn], [uo1, uo2, ..., uon]]
# training [batch_size, initial_learning_rate]

In [ ]:
prediction_results_df = pd.DataFrame(\
                                     columns=['model_id', 'counter', 'dataset', 'string_timestamp', 'prediction', 'target'])

In [ ]:
for target in targets:
    # use only 'TPU_XX' as model identifier
    model_id = target[0][-6:]
    for counter in np.arange(target[1]):
        for dataset in ['test', 'eval']:
            # path to json files in stats/ directory            
            path = 'stats/{}_{:02d}/prediction_results_on_{}_dataset.json'.format(model_id, counter, dataset)
            # load json file to dictionary
            with open(path, 'r') as json_file:
                prediction_results = json.load(json_file)
            # how many predictions in the dataset?
            length = len(prediction_results['predictions'])
            # a list with model_id repeated length times
            model_id_repeat_list = [model_id]*length
            # same for counter
            counter_repeat_list = [counter]*length
            # same for dataset
            dataset_repeat_list = [dataset]*length
            # buffer dataframe
            df = pd.DataFrame(list(zip(model_id_repeat_list,
                                       counter_repeat_list,
                                       dataset_repeat_list,
                                       prediction_results['string_timestamps'],
                                       prediction_results['predictions'],
                                       prediction_results['targets'])), columns = columns)
            # append buffer to final dataframe
            prediction_results_df = prediction_results_df.append(df, ignore_index=True)

In [ ]:
### persist the Pandas dataframe to avoid re-calculating
prediction_results_df.to_pickle('../database/prediction_results_TPU_10_to_TPU_52.pkl')

In [4]:
# start here to read the persisted dataframe
prediction_results_TPU_10_to_TPU_52_df = pd.read_pickle('../database/prediction_results_TPU_10_to_TPU_52.pkl')

In [5]:
prediction_results_TPU_10_to_TPU_52_df

,model_id,counter,dataset,string_timestamp,prediction,target
0,TPU_10,0,test,2018-02-23 14:00:00,10.342652,6.213650
1,TPU_10,0,test,2018-02-23 15:00:00,10.358902,8.161867
2,TPU_10,0,test,2018-02-23 16:00:00,8.927979,6.752000
3,TPU_10,0,test,2018-02-23 17:00:00,7.095260,5.244833
4,TPU_10,0,test,2018-02-23 18:00:00,6.992952,7.247250
...,...,...,...,...,...,...
402114,TPU_52,19,eval,2018-01-26 09:00:00,6.207547,7.063433
402115,TPU_52,19,eval,2018-01-26 10:00:00,7.115816,6.017716
402116,TPU_52,19,eval,2018-01-26 11:00:00,4.591192,7.312567
402117,TPU_52,19,eval,2018-01-26 12:00:00,7.437450,5.746467


In [9]:
# a demo on dataframe group by to get aggregated statistics
prediction_results_df.groupby(['model_id', 'counter', 'dataset']).mean()

prediction    target
model_id counter dataset                      
TPU_10   0       eval       4.828188  4.592749
                 test       6.372000  6.183965
         1       eval       4.825500  4.592749
                 test       6.342292  6.183965
         2       eval       5.033116  4.592749
...                              ...       ...
TPU_52   17      test       6.359495  6.183965
         18      eval       4.905027  4.592749
                 test       6.481611  6.183965
         19      eval       4.963964  4.592749
                 test       6.352068  6.183965

[758 rows x 2 columns]

In [ ]:
# build and test the SMAPE function

predictions = np.array([0.9, 2.1, 3.1, 3.9, 5.1, 6.1, 7.1, 8.1, 9.1, 10.1])

targets = np.array([1., 2., 3., 4., 5., 6., 7., 8., 9., 10.])

np.abs(predictions - targets)

np.abs(predictions) +  np.abs(targets)

2.*np.abs(predictions - targets) / (np.abs(predictions) +  np.abs(targets))

np.sum(2.*np.abs(predictions - targets) / (np.abs(predictions) +  np.abs(targets)))/predictions.shape[0]

In [10]:
# symmetrical mean absolute percentage error
def smape(predictions, targets):
    '''
    predictions: a NumPy array with the predicted values
    targets: a NumPy array with the actual values
    '''
    # verify predictions and targets have the same shape
    if predictions.shape == targets.shape:
            return(np.sum(2*np.abs(predictions - targets) /
                          (np.abs(targets) + np.abs(predictions)))/predictions.shape[0])

In [11]:
# use the code in previous cells and iterate on prediction_results_df to build statistics

In [17]:
# an empty dataframe to store smape values per model execution
smape_values_df = pd.DataFrame(columns=['model_id', 'counter', 'dataset', 'smape'])

In [18]:
for target in targets:
    model_id = target[0][-6:]
    for counter in np.arange(target[1]):
        for dataset in ['test', 'eval']:
            # get a boolean to filter prediction_results_df on the model_id, counter, and dataset
            flag = \
            prediction_results_df.model_id.eq(model_id) & \
            prediction_results_df.counter.eq(counter) & \
            prediction_results_df.dataset.eq(dataset)
            
            filtered_df = prediction_results_df[flag]
            # a row dataframe to calculate smape for a given model_id, counter, and dataset
            row_df = pd.DataFrame([[model_id,
                                    counter,
                                    dataset,
                                    smape(filtered_df.prediction, filtered_df.target)]],
                                  columns=['model_id', 'counter', 'dataset', 'smape'])
            
            smape_values_df = smape_values_df.append(row_df, ignore_index=True)

In [19]:
smape_values_df

,model_id,counter,dataset,smape
0,TPU_10,0,test,0.185395
1,TPU_10,0,eval,0.294091
2,TPU_10,1,test,0.181967
3,TPU_10,1,eval,0.306450
4,TPU_10,2,test,0.182526
...,...,...,...,...
753,TPU_52,17,eval,0.281668
754,TPU_52,18,test,0.176599
755,TPU_52,18,eval,0.289574
756,TPU_52,19,test,0.163662


In [20]:
# persist Pandas dataframe, to avoid re-calculating
smape_values_df.to_pickle('../database/smape_values_TPU_10_to_TPU_52.pkl')

In [9]:
# once persisted, the dataframe can be recovered here
smape_values_TPU_10_to_TPU_52_df = pd.read_pickle('../database/smape_values_TPU_10_to_TPU_52.pkl')

In [12]:
smape_values_df[:6]

,model_id,counter,dataset,smape
0,TPU_10,0,test,0.185395
1,TPU_10,0,eval,0.294091
2,TPU_10,1,test,0.181967
3,TPU_10,1,eval,0.306450
4,TPU_10,2,test,0.182526
5,TPU_10,2,eval,0.300744


In [ ]:
# smape_values_df.groupby(['model_id', 'dataset'])[['smape']].mean()

In [18]:
# recover statistics for the dataset that renders the best predictive performance
is_test = smape_values_df.dataset.eq('test')
smape_values_df[is_test].groupby(['model_id']).describe()

smape                                                              \
         count      mean       std       min       25%       50%       75%   
model_id                                                                     
TPU_10    10.0  0.192778  0.012227  0.181967  0.185416  0.187326  0.199728   
TPU_11    10.0  0.192240  0.013084  0.170127  0.181629  0.193835  0.203189   
TPU_13    10.0  0.195072  0.026210  0.167397  0.180900  0.190723  0.200412   
TPU_14    10.0  0.185609  0.011031  0.170653  0.177081  0.183679  0.190324   
TPU_15    10.0  0.198589  0.009300  0.183377  0.194047  0.196849  0.203721   
TPU_16    10.0  0.189259  0.010594  0.169699  0.182343  0.189767  0.197696   
TPU_17    10.0  0.190017  0.011908  0.167305  0.185008  0.189198  0.199454   
TPU_18    10.0  0.185414  0.007599  0.174830  0.180908  0.183616  0.190030   
TPU_19    10.0  0.187095  0.011836  0.169975  0.180422  0.186846  0.192534   
TPU_20    10.0  0.179525  0.012984  0.153597  0.175997  0.180900  0.185266   
TPU_21    10.0  0.182127  0.009645  0.172585  0.176381  0.180055  0.182020   
TPU_22    10.0  0.185894  0.006853  0.171582  0.184362  0.186218  0.188385   
TPU_23    10.0  0.176622  0.007510  0.167981  0.170406  0.176847  0.178470   
TPU_24    10.0  0.175934  0.007149  0.166603  0.169687  0.176062  0.182050   
TPU_25    10.0  0.178262  0.005115  0.172757  0.175369  0.177489  0.178902   
TPU_26    10.0  0.178654  0.007309  0.167256  0.172682  0.179509  0.184934   
TPU_27    10.0  0.180361  0.013610  0.167765  0.172968  0.176774  0.179719   
TPU_30    10.0  0.177242  0.006142  0.169039  0.173449  0.176013  0.178536   
TPU_31    10.0  0.174230  0.009967  0.160518  0.167040  0.174247  0.178520   
TPU_32    10.0  0.177531  0.006812  0.169548  0.173041  0.175242  0.181389   
TPU_33    10.0  0.182014  0.007735  0.171450  0.178642  0.180707  0.186359   
TPU_34    10.0  0.178195  0.005520  0.168718  0.175559  0.179235  0.181677   
TPU_35    10.0  0.363743  0.575010  0.169519  0.175130  0.180624  0.188745   
TPU_36     9.0  0.179369  0.006252  0.172755  0.172917  0.183354  0.183695   
TPU_37    10.0  0.368661  0.573263  0.175380  0.179734  0.190656  0.195618   
TPU_38    10.0  0.174938  0.007870  0.164127  0.168649  0.176271  0.178947   
TPU_39    10.0  0.174667  0.009931  0.157136  0.169632  0.173912  0.181350   
TPU_40    10.0  0.177885  0.006236  0.163790  0.174800  0.179882  0.180645   
TPU_41    10.0  0.186919  0.027607  0.167714  0.171871  0.180253  0.187138   
TPU_42    10.0  0.183791  0.007593  0.171102  0.181905  0.184351  0.189858   
TPU_50    30.0  0.180694  0.006768  0.164966  0.175567  0.180742  0.186419   
TPU_51    30.0  0.175773  0.007030  0.162726  0.173272  0.176514  0.178696   
TPU_52    20.0  0.171641  0.006667  0.160474  0.167405  0.170323  0.176862   

                    
               max  
model_id            
TPU_10    0.214226  
TPU_11    0.208713  
TPU_13    0.261940  
TPU_14    0.203853  
TPU_15    0.215191  
TPU_16    0.203950  
TPU_17    0.207264  
TPU_18    0.198833  
TPU_19    0.211095  
TPU_20    0.199303  
TPU_21    0.203987  
TPU_22    0.195512  
TPU_23    0.192678  
TPU_24    0.186353  
TPU_25    0.190049  
TPU_26    0.188685  
TPU_27    0.215034  
TPU_30    0.187768  
TPU_31    0.191234  
TPU_32    0.191370  
TPU_33    0.194856  
TPU_34    0.185656  
TPU_35    2.000000  
TPU_36    0.186781  
TPU_37    2.000000  
TPU_38    0.189606  
TPU_39    0.191262  
TPU_40    0.186421  
TPU_41    0.262150  
TPU_42    0.192489  
TPU_50    0.192439  
TPU_51    0.189953  
TPU_52    0.183872